In [1]:
import cv2
import numpy as np
import open3d as o3d
import json
import glob
import matplotlib.pyplot as plt
import copy
import teaserpp_python
from numpy.linalg import inv
from scipy.spatial import cKDTree
import time
from PIL import Image
import torchvision
import torch
from torchvision import transforms as T 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from numba import jit, prange
import os
import csv 
# Load MaskRCNN 

from multiprocessing import Pool

def pad_depth_to_color(depth, color):
    # Calculate differences in dimensions
    height_diff = color.shape[0] - depth.shape[0]
    width_diff = color.shape[1] - depth.shape[1]
    
    # Compute padding values for height and width
    pad_top = height_diff // 2
    pad_bottom = height_diff - pad_top
    pad_left = width_diff // 2
    pad_right = width_diff - pad_left
    
    # Apply padding
    padded_depth = np.pad(depth, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    
    return padded_depth

def load_maskrcnn_model(model_path):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, 2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def get_model_mask(model_generic, image):
    proba_threshold = 0.5
    ig = transform(image)
    with torch.no_grad():
        prediction = model_generic([ig.to(device)])
        
    if(prediction[0]['masks'].nelement() == 0):
        XX = torch.empty((0,0), dtype=torch.int64)
        return XX
    predicted_mask = prediction[0]
    predicted_mask = predicted_mask['masks'][0] > proba_threshold
    
    predicted_mask = predicted_mask.squeeze(1)
    mask = predicted_mask.cpu().detach()
    return mask


def segment_images_modified(last_frame):
    depth_PIL = Image.fromarray(np.asarray(last_frame.depth)).convert("RGB")
    rgb_image_array = np.asarray(last_frame.color)
    depth_image_array = np.asarray(last_frame.depth)
    rgb_PIL = Image.fromarray(rgb_image_array)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    rgb_mask = get_model_mask(model_rgb, rgb_PIL)
    depth_mask = get_model_mask(model_depth, depth_PIL)

    if depth_mask.nelement() == 0:
        mask_combined = rgb_mask
    else:
        mask_combined = depth_mask | rgb_mask  # 1 vote arbitration (OR the masks)

    # Convert tensor to numpy array and ensure the right datatype
    mask_combined = mask_combined.numpy().astype(rgb_image_array.dtype)
    mask_image = mask_combined.swapaxes(0, 2).swapaxes(0, 1)
    mask_image = (mask_image > 0).astype(rgb_image_array.dtype)   

    fg_image_rgb = rgb_image_array * mask_image

    # For the depth image:
    squeezed_mask = np.squeeze(mask_image)
    fg_image_depth = depth_image_array * squeezed_mask
    
    
    last_frame.color = o3d.geometry.Image(fg_image_rgb)
    last_frame.depth = o3d.geometry.Image(fg_image_depth)
   


    return last_frame


def filter_file_names(file_list): # to avoid the two head cameras
    new_file_list = []
    for f in file_list:
        tmp = f.split('.mkv')[0]
        ID  = tmp.split('_')[-1]
        if (ID == "13" or ID == "15"):
            continue
        else :
            new_file_list.append(f)
    return new_file_list


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def cluster_point_cloud_new(outlier_cloud):
    cloud_colors = copy.deepcopy(np.asarray(outlier_cloud.colors).T)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=False))

    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    # Filter points, normals, and colors for the largest cluster
    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals = pcd2normals(outlier_cloud)

    cloud_filtered = cloud_xyz[:, labels == largest_cluster_label]
    normals_filtered = cloud_normals[:, labels == largest_cluster_label]
    colors_filtered = cloud_colors[:, labels == largest_cluster_label]

    # Create a point cloud for the largest cluster
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)

    #o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster

def upsample_using_reference_normals(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    """
    Efficiently upsample the sparse point cloud using the dense point cloud as reference.
    Considers the normals to ensure added points are consistent with the sparse cloud.

    Parameters:
    - sparse_pcd: The sparse point cloud
    - dense_pcd: The dense reference point cloud
    - search_radius: Radius to search for neighbors
    - angle_threshold: Maximum angle in degrees between normals to consider a point

    Returns:
    - A new upsampled point cloud
    """
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    kdtree = o3d.geometry.KDTreeFlann(dense_pcd)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    added_points = set(map(tuple, sparse_points))

    upsampled_points = list(sparse_points)

    for i, point in enumerate(sparse_points):
        [k, idx, _] = kdtree.search_radius_vector_3d(point, search_radius)
        
        neighbors = np.asarray(dense_pcd.points)[idx]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbors = neighbors[angles < angle_threshold]

        # Filtering out points that are already in the sparse cloud or have been added before
        unique_valid_neighbors = [tuple(neighbor) for neighbor in valid_neighbors if tuple(neighbor) not in added_points]

        upsampled_points.extend(unique_valid_neighbors)
        added_points.update(unique_valid_neighbors)

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd


def pcd2normals(pcd):
    return np.asarray(pcd.normals).T
def pcd2xyz(pcd):
    return np.asarray(pcd.points).T


@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    return result

def colored_ICP(source, target):
    
    voxel_radius = [0.04, 0.02, 0.01]
    max_iter = [50, 30, 14]
    current_transformation = np.identity(4)
    #print("3. Colored point cloud registration")
    for scale in range(3):
        iters = max_iter[scale]
        radius = voxel_radius[scale]
        #print("iteration: ", iters, radius, scale)

        #print("3-1. Downsample with a voxel size %.2f" % radius)
        source_down = copy.deepcopy(source).voxel_down_sample(radius)
        target_down = copy.deepcopy(target).voxel_down_sample(radius)

        #print("3-2. Estimate normal.")
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))

        #print("3-3. Applying colored point cloud registration")
        result_icp = o3d.pipelines.registration.registration_colored_icp(
            source_down, target_down, radius, current_transformation,
            o3d.pipelines.registration.TransformationEstimationForColoredICP(),
            o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                              relative_rmse=1e-6,
                                                              max_iteration=iters))
        current_transformation = result_icp.transformation
    
   
        #draw_registration_result(source, target, current_transformation)
    return current_transformation

def backproject_o3d(rgbd_frame, intrinsics):
    
    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgbd_frame.color, rgbd_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc, intrinsics)
    n_radius = 0.01*2.0
    pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=n_radius, max_nn=30))
    
    return pcd

def load_calibration(tform_path, num_transforms=5):
    transforms = [np.eye(4)]  # Initialize with identity matrix
    
    # Load transformations from file
    for i in range(1, num_transforms+1):
        filename = tform_path + f"H_0_{i}.txt"
        
        if not os.path.exists(filename):
            raise FileNotFoundError(f"The file {filename} does not exist!")
        
        transforms.append(np.loadtxt(filename))
    
    return transforms

def perform_pairwise_alignment(pcds_tsdf,pcds_cropped):
    #"""Compute and apply transformations."""
    t01 = colored_ICP(pcds_tsdf[0], pcds_tsdf[1])
    t52 = colored_ICP(pcds_tsdf[5], pcds_tsdf[2])
    t43 = colored_ICP(pcds_tsdf[4], pcds_tsdf[3])
    t12 = colored_ICP(pcds_tsdf[1], pcds_tsdf[2])
    t32 = colored_ICP(pcds_tsdf[3], pcds_tsdf[2])

    H0 = t12 @ t01
    H1 = t12
    H3 = t32
    H4 = t32 @ t43
    H5 = t52

    # Transform the point clouds
    p0 = copy.deepcopy(pcds_cropped[0]).transform(H0)
    p1 = copy.deepcopy(pcds_cropped[1]).transform(H1)
    p3 = copy.deepcopy(pcds_cropped[3]).transform(H3)
    p4 = copy.deepcopy(pcds_cropped[4]).transform(H4)
    p5 = copy.deepcopy(pcds_cropped[5]).transform(H5)
    
    d0 = copy.deepcopy(pcds_tsdf[0]).transform(H0)
    d1 = copy.deepcopy(pcds_tsdf[1]).transform(H1)
    d3 = copy.deepcopy(pcds_tsdf[3]).transform(H3)
    d4 = copy.deepcopy(pcds_tsdf[4]).transform(H4)
    d5 = copy.deepcopy(pcds_tsdf[5]).transform(H5)
    
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = p0+p1+pcds_cropped[2]+p3+p4+p5
    
    ptsdf_combined = o3d.geometry.PointCloud()
    ptsdf_combined = d0+d1+pcds_tsdf[2]+d3+d4+d5

    return pcd_combined, ptsdf_combined

def process_file(i):
    inFile = files[i]
    fname = inFile.split('/')[-1]
    file_name = fname.split('.mkv')[0]
    
    reader = o3d.io.AzureKinectMKVReader()
    reader.open(inFile)
    
    if not reader.is_opened():
        raise RuntimeError("Unable to open file {}".format(inFile))
    
    metadata = reader.get_metadata()

    # write the metadata to a JSON file since that seems to be the only
    # way to retrieve that data
    o3d.io.write_azure_kinect_mkv_metadata('{}/{}_intrinsic.json'.format(abspath, file_name), metadata)

    # Open the file and load the JSON
    with open(abspath + "/" + file_name + "_intrinsic.json") as f:
        data = json.load(f)
    
    height = data['height']
    width = data['width']
    intrinsics = data["intrinsic_matrix"]
    time_stamp = data["stream_length_usec"]

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    cx = intrinsics[6]
    cy = intrinsics[7]
    fx = intrinsics[0]
    fy = intrinsics[4]
    camera_intrinsics.set_intrinsics(width, height, fx, fy, cx, cy)

    last_frame = None
    while not reader.is_eof():
        rgbda = reader.next_frame()
        if rgbda is None:
            continue
        last_frame = rgbda

    if last_frame is not None:
        return last_frame
    else:
        print("************No valid frames found in the .mkv file.**********")
        return None

def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)

    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    
    # This retrieves the indices of all neighbors within the search_radius
    neighbor_indices = dense_tree.query_ball_point(sparse_points, search_radius, workers=-1)
    
    valid_indices = []
    for i, idx_row in enumerate(neighbor_indices):
        neighbors = dense_points[idx_row]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]

        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        # Filtering neighbors based on angle threshold
        valid_neighbor_indices = np.array(idx_row)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
#     valid_indices = []
#     for i, idx_row in enumerate(neighbor_indices):
#         neighbors = dense_points[idx_row]
#         neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]
        
#         # Calculate angles between sparse point's normal and all its neighbors' normals
#         angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

#         # Filtering neighbors based on angle threshold
#         valid_for_current = idx_row[angles < angle_threshold]
#         valid_indices.extend(valid_for_current)

    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    
    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.estimate_normals()

    return upsampled_pcd


def load_filter_pcds(data_path, ctform):  # returns a list of  pcds
    ws = 2
    flying_pixel_filter_threshold = 1000

    reader = o3d.io.AzureKinectMKVReader()
    abspath = data_path
    pcds = []
    files_raw = glob.glob(data_path+'/*.mkv')
    files = filter_file_names(files_raw)
    files.sort()   

   
    list_size = len(files)
    rgbd_frames = [None] * list_size
    pcds_tsdf = []
    
    l1 = time.time()
    for i in range(len(files)): # for each view
   
        inFile = files[i]
        fname = inFile.split('/')[-1]
        file_name = fname.split('.mkv')[0]
        reader.open(inFile)
        if not reader.is_opened():
            raise RuntimeError("Unable to open file {}".format(inFile))
        metadata = reader.get_metadata()
  
        # write the metadata to a JSON file since that seems to be the only
        # way to retrieve that data
        o3d.io.write_azure_kinect_mkv_metadata(
                    '{}/{}_intrinsic.json'.format(abspath,file_name), metadata)

        # Open the file and load the JSON
        with open(abspath+"/" + file_name + "_intrinsic.json") as f:
            data = json.load(f)
        
        height = data['height']
        width = data['width']
        intrinsics = data["intrinsic_matrix"]
        time_stamp = data["stream_length_usec"]
        #print(f"Intrinsic Matrix {intrinsics}")

        camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
        cx = intrinsics[6]
        cy = intrinsics[7]
        fx = intrinsics[0]
        fy = intrinsics[4]
        camera_intrinsics.set_intrinsics(width,height,fx,fy,cx,cy)

        last_frame = None
        while not reader.is_eof(): # go until hitting eof because of exposure issues in early color frames
            rgbda = reader.next_frame();
            if rgbda is None:
                #print("Got nothing! ")
                continue
            last_frame = rgbda

        if last_frame is not None:
            #print("Got the last frame")
            rgbd_frames[i] = last_frame
        else:
            
            print("************No valid frames found in the .mkv file.**********")
    
        # filter the depth image for flying pixels
        depth_image_array = np.asarray(last_frame.depth)
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        last_frame.color, last_frame.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        volume = o3d.pipelines.integration.ScalableTSDFVolume(
                    voxel_length= 4.0/ 512.0,
                    sdf_trunc=0.4,
                    color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

        volume.integrate(
            rgbdc, # rgbdc originally - trying with masked_rgbd
            camera_intrinsics,
            np.eye(4),
        )
#         pcd_raw = backproject_o3d(last_frame,camera_intrinsics)
        pcd_tsdf = volume.extract_point_cloud()
        # First eliminate flying pixels before integrating the depth map
        result_mask = numba_eliminate_flying_pixels(depth_image_array.copy(), ws, flying_pixel_filter_threshold)
        depth_image_array[result_mask > flying_pixel_filter_threshold] = 0
        # Re-insert filtered depth into the rgbd image
        last_frame.depth = o3d.geometry.Image(depth_image_array)
      
        # Apply maskrcnn to filtered depth image
        masked_rgbd = segment_images_modified(last_frame,i,data_path)
        # necessary lol
        rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
        pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc,
                                                            camera_intrinsics)
      
        #print("Time was: ", w2-w1, " s")
        pcds.append(copy.deepcopy(pcd_tmp).transform(np.eye(4)))
        pcds_tsdf.append(copy.deepcopy(pcd_tsdf).transform(np.eye(4)))
        reader.close()
    l2 = time.time()  
    print("Loading files: ", l2-l1, " s")
    return pcds,pcds_tsdf


def save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v):
    # Save the mesh and PCD
    mesh_filename = os.path.join(data_path, f"{animal_id}_mesh.ply")
    pcd_filename = os.path.join(data_path, f"{animal_id}_pcd_downsampled.ply")

    o3d.io.write_triangle_mesh(mesh_filename, mesh)
    
    o3d.io.write_point_cloud(pcd_filename, pcd_downsampled)

    CSV_PATH = '/home/vigir3d/Datasets/cattle_scans/cow_measurements.csv'
    # Check if CSV file exists to decide whether to write headers
    write_header = not os.path.exists(CSV_PATH)
  # Save SA & V to the CSV file in append mode
    with open(CSV_PATH, 'a', newline='') as csvfile:
        fieldnames = ['Animal ID', 'SA', 'V']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if write_header:
            writer.writeheader()

        writer.writerow({'Animal ID': animal_id, 'SA': sa, 'V': v})
        
def cluster_point_cloud_tensor(pcd) :
    colors = pcd.point.colors
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Error) as cm:
        labels = pcd.cluster_dbscan(eps=0.1, min_points=10, print_progress=False)
        
    
    labels = labels.cpu()
    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    colors[labels < 0] = 0
    pcd.point.colors = colors
    return pcd


# Main
rgb_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_v2.pth'
depth_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_depth_best.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loads the maskrcnn trained for depth and rgb
model_rgb = load_maskrcnn_model(rgb_model_path)
model_depth = load_maskrcnn_model(depth_model_path)


transform = T.ToTensor()
# These two will need to be requested from command line
# -i   | input data path
# - t  | calibration data path 
data_path = '/home/vigir3d/Datasets/cattle_scans/Cattle_scan_11_17_22/Animal_1_1'
tform_path = '/home/vigir3d/Datasets/cattle_scans/farm_07_24/Animal_calib/'

if __name__ == "__main__":
    # Run things
    tensor = False
    if  tensor :
        s = time.time()

        l1 = time.time()
        tforms = load_calibration(tform_path,5)
        pcds, ptsdf = load_filter_pcds(data_path,tforms)
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        l2 = time.time()

        #place on the GPU
        # Tensorize
        # get the Device
        p1 = time.time()
        o3d_device = o3d.core.Device("CUDA:0") 
        # convert segmented and full clouds to tensor
        pcd_t = o3d.t.geometry.PointCloud.from_legacy(pcd_all).to(o3d_device)
        #pcd_t_tsdf = o3d.t.geometry.PointCloud.from_legacy(ptsdf_all).to(o3d_device)
        pcd_t.estimate_normals()
        # Perform clustering to get largest cloud of points from segmented cloud
        pcd_clustered = cluster_point_cloud_tensor(pcd_t)
        # place on cpu
        pcd_clustered_cpu = pcd_clustered.cpu().to_legacy()
        p2 = time.time()

        u1 = time.time()
        # Upsample the single cluster cloud using the full cloud as reference
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered_cpu, ptsdf_all)
        u2 = time.time()

        tn1 = time.time()
        # Downsample uniformly to ease compute
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        # Correct for the normal orientation problem
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        tn2 = time.time()

        m1 = time.time()
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
                 mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)
        m2 = time.time()
        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        animal_id = os.path.basename(os.path.normpath(data_path))
        save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)
        e = time.time()

        print("Total time: ", e-s, " s")


        print("Load and register: ", l2-l1, " s")

        print("Preprocess: ", p2-p1, " s")

        print("Upsampling: ", u2-u1, " s")
        print("Normal orientation: ", tn2-tn1, " s")    
        print("Meshing: ", m2-m1, " s")
    
    elif tensor :
        # Run things
        s = time.time()

        l1 = time.time()
        tforms = load_calibration(tform_path,5)
        pcds, ptsdf = load_filter_pcds(data_path,tforms)
        l2 = time.time()
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        r2 = time.time()
        
        p1 = time.time()
        pcd_all.estimate_normals()
        # cluster first to remove extra noise
        pcd_clustered = cluster_point_cloud_new(pcd_all)
        # After clustering, we upsample from the initial
        p2 = time.time()

        u1 = time.time()
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
        u2 = time.time()

        tn1 = time.time()
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        #pcd_downsampled.orient_normals_towards_camera_location()
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        tn2 = time.time()

        m1 = time.time()
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
                mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        m2 = time.time()
        animal_id = os.path.basename(os.path.normpath(data_path))
        save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)

        e = time.time()
        print("Load files: ", l2-l1, " s")
        print("Register clouds: ", r2-l2, " s")

        print("Remove extra points: ", p2-p1, " s")

        print("Upsampling: ", u2-u1, " s")
        print("Normal orientation: ", tn2-tn1, " s")    
        print("Meshing: ", m2-m1, " s")
        print("Total time: ", e-s, " s")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#dpath = '/home/vigir3d/Datasets/cattle_scans/farm_scan1/Animal_482_2/'
def process_file_list_comp(file_id, dpath):
    ws = 2
    flying_pixel_filter_threshold = 0.1
    # Format filenames
    dpath = dpath.rstrip('/')

    # Split by the directory separator and get the last part
    suffix = os.path.basename(dpath)
    
    file_suffix = suffix + "_nano_" + str(file_id)
    # Construct full file paths using os.path.join for better cross-platform compatibility
    depth_file = os.path.join(dpath, file_suffix + "_depth.png")
    color_file = os.path.join(dpath, file_suffix + ".jpg")
    intrinsics_file = os.path.join(dpath, file_suffix + "_intrinsics.txt")
    # Read the files
    depth = o3d.io.read_image(depth_file)
    color = o3d.io.read_image(color_file)
    #depth = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11_depth.png')
    #color = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11.jpg')
    K = np.loadtxt(intrinsics_file)
      

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsics.set_intrinsics(int(K[0]), int(K[1]), K[2], K[3], K[4], K[5])

    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False
    )

    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=4.0 / 512.0, sdf_trunc=0.04, color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
    )

    volume.integrate(rgbdc, camera_intrinsics, np.eye(4))
    pcd_tsdf = volume.extract_point_cloud()
    depth_np = np.asarray(rgbdc.depth)
    result_mask = numba_eliminate_flying_pixels(depth_np.copy(), ws, flying_pixel_filter_threshold)
    depth_np[result_mask > flying_pixel_filter_threshold] = 0
    # Re-insert filtered depth into the rgbd image
    rgbdc.depth = o3d.geometry.Image(depth_np)
    # Apply maskrcnn to filtered depth image
    masked_rgbd = segment_images_modified(rgbdc)
    # necessary lol
    rgbdc_new = o3d.geometry.RGBDImage.create_from_color_and_depth(
                    masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc_new,
                                                        camera_intrinsics)
    return pcd_tsdf, pcd_tmp 


if __name__ == "__main__":
    dpath = '/home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_1_1/'
    file_ids = [11,12,14,16,17,18]  # exclude head cameras

    # Generate a list of volumes using list comprehension
    start = time.time()
    results = [process_file_list_comp(file_id, dpath) for file_id in file_ids]
    ptsdf, pcds = zip(*results)
    l = False
    if l :
        pcd_all, ptsdf_all = perform_pairwise_alignment(ptsdf,pcds)
        pcd_all.estimate_normals()
        pcd_clustered = cluster_point_cloud_new(pcd_all)
        pcd_upsampled = upsample_using_reference_normals_new(pcd_clustered, ptsdf_all)
        pcd_downsampled = pcd_upsampled.voxel_down_sample(0.01)
        pcd_downsampled.orient_normals_consistent_tangent_plane(30)
        with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_downsampled, depth=6)

        sa = mesh.get_surface_area()
        if(mesh.is_watertight()):
            #print("Is watertight 1")
            v =  mesh.get_volume()
        else : 
            v = 0.0
        animal_id = os.path.basename(os.path.normpath(data_path))
        #save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v)


    end = time.time()

    print("Duration was:", end-start, "s")

FileNotFoundError: /home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_1_1/Animal_1_1_nano_11_intrinsics.txt not found.

In [ ]:
dpath = '/home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_1_1/'